In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
titanic_df = pd.read_csv('data/titanic.csv')
titanic_df.head()

In [ ]:
titanic_df.info()

In [ ]:
titanic_df.describe()

In [ ]:
titanic_df.isnull().sum()

In [ ]:
#하수
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())

In [ ]:
titanic_df.groupby(['Pclass','Sex'])['Age'].size()

In [ ]:
titanic_df.groupby(['Pclass','Sex'])['Age'].median()

In [ ]:
titanic_df['Age'] = (
    titanic_df
    .groupby(['Pclass','Sex'])['Age']
    .transform(lambda x: x.fillna(x.median()))
    .astype(int)
)

In [ ]:
titanic_df.isnull().sum()

In [ ]:
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('N')
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('N')

In [ ]:
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]

In [ ]:
titanic_df.head()

In [ ]:
#이상치 체크
  
num_df = titanic_df.select_dtypes(include='number').columns
num_df

In [ ]:
for col in num_df:
    Q1 = titanic_df[col].quantile(0.25)
    Q3 = titanic_df[col].quantile(0.75)
    IQR = Q3-Q1
    lower = Q1 - 1.5 *  IQR
    upper = Q3 + 1.5 *  IQR

    print(f'{col}이상치갯수:')
    print(lower)
    print(upper)

In [ ]:
sns.histplot(titanic_df['Fare'])

In [ ]:
titanic_df['Fare'] = titanic_df['Fare'].clip(0,300)

In [ ]:
sns.histplot(titanic_df['Fare'])

In [ ]:
sns.histplot(titanic_df['Age'])

In [ ]:
sns.countplot(x='Sex',data=titanic_df)

In [ ]:
sns.barplot(x='Sex',y='Survived',data=titanic_df)

In [ ]:
sns.barplot(x='Pclass',y='Survived',data=titanic_df)

In [ ]:
sns.barplot(x='Pclass',y='Survived',hue='Sex',data=titanic_df)

In [ ]:
df = titanic_df.drop(['PassengerId','Name','Ticket'],axis=1)

In [ ]:
df.head()

In [ ]:
df[['Sex','Cabin','Embarked']]

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Sex'] = le.fit_transform(df['Sex'])
df['Cabin'] = le.fit_transform(df['Cabin'])
df['Embarked'] = le.fit_transform(df['Embarked'])

df.head()

In [ ]:
df.drop(['Survived'],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df.drop(['Survived'],axis=1),df['Survived'],stratify=df['Survived'],random_state=3333)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier(random_state=3333)
grid_param = {'max_depth':[2,3,4,5,6],'min_samples_split':[2,3,4,5,6,7,8,9]}

grid_model = GridSearchCV(model, param_grid=grid_param, cv=5)

grid_model.fit(X_train,y_train)



In [ ]:
grid_model.best_score_

In [ ]:
pred = grid_model.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_test)